In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from pathlib import Path


data_dir = "/kaggle/input/ka-ocr"
for item in Path(data_dir).iterdir():
    print(item)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ka-ocr/bpg_phone_sans_bold_italic
/kaggle/input/ka-ocr/bpg_glaho_bold
/kaggle/input/ka-ocr/bpg_irubaqidze
/kaggle/input/ka-ocr/bpg_boxo-boxo
/kaggle/input/ka-ocr/bpg_glaho_2008
/kaggle/input/ka-ocr/bpg_paata_cond
/kaggle/input/ka-ocr/mg_bitneon_chaos
/kaggle/input/ka-ocr/literaturulitt
/kaggle/input/ka-ocr/ar-archy-regular
/kaggle/input/ka-ocr/bpg_paata_caps
/kaggle/input/ka-ocr/mg_bitneon
/kaggle/input/ka-ocr/fixedsys_excelsior
/kaggle/input/ka-ocr/ka_literaturuli
/kaggle/input/ka-ocr/gugeshashvili_slfn_2
/kaggle/input/ka-ocr/bpg_glaho
/kaggle/input/ka-ocr/bpg_dedaena
/kaggle/input/ka-ocr/NotoSansGeorgian
/kaggle/input/ka-ocr/gf_aisi_nus-bold-italic
/kaggle/input/ka-ocr/arial_geo-bold-italic
/kaggle/input/ka-ocr/arial_geo-italic
/kaggle/input/ka-ocr/bpg_sans_2008
/kaggle/input/ka-ocr/bpg_nostalgia
/kaggle/input/ka-ocr/bpg_supersquare_2009
/kaggle/input/ka-ocr/arial_geo
/kaggle/input/ka-ocr/bpg_phone_sans_bold
/kaggle/input/ka-ocr/alkroundednusx-medium
/kaggle/input/ka-oc